In [22]:
import snowflake.connector
import os
import pyodbc
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup
import numpy as np


account = 'ffa72186.us-east-1'

user = 'B042304@MERCANTIL.COM.BR'
password = 'trabalho@03T'

warehouse = 'WH_DEV'
database = 'SDX_GERENCIA_CRESCIMENTO'
schema = 'CONHECIMENTO_CLIENTE_BRONZE'
authenticator = 'externalbrowser'

conn_snow = snowflake.connector.connect(
    user=user,
    authenticator=authenticator,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

cur_snow = conn_snow.cursor()

In [5]:
response = requests.get('https://www.cora.com.br/blog/codigos-bancos/')
html = response.text

In [6]:
soup = BeautifulSoup(html, 'html.parser')
tables = soup.find_all('table')

In [39]:
data = []
rows = soup.find_all('tr')

for row in rows:
    columns = row.find_all('td')
    data_row = [column.get_text(strip=True) for column in columns]
    data.append(data_row)

df_info = pd.DataFrame(data, columns=['Codigo', 'Banco', 'ISPB'])
df_info = df_info.loc[df_info['Banco'] != 'Banco']
df_info = df_info.loc[df_info['Banco'] != 'n/a']
df_info = df_info.loc[df_info['Banco'] != '']

df_info

,Codigo,Banco,ISPB
0,0,001,Banco do Brasil S.A.
1,208,070,BRB – BANCO DE BRASILIA S.A.
4,250699,272,AGK CORRETORA DE CAMBIO S.A.
5,315557,136,CONFEDERAÇÃO NACIONAL DAS COOPERATIVAS CENTRAI...
6,360305,104,CAIXA ECONOMICA FEDERAL
...,...,...,...
266,92856905,117,ADVANCED CORRETORA DE CÂMBIO LTDA
267,92874270,654,BANCO DIGIMAIS S.A.
268,92875780,371,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...
269,92894922,212,Banco Original S.A.


In [40]:
# df_info['Comprimento'] = df_info['Banco'].str.len()
# df_info.max()

query = """
    CREATE OR REPLACE TABLE SDX_GERENCIA_CRESCIMENTO.CONHECIMENTO_CLIENTE_SILVER.B042304_TI_BANCOS_BRASIL (
        CODIGO_ISPB INT,
        CODIGO_BANCO INT,
        NOM_BANCO VARCHAR(100)
    )
"""

cur_snow.execute(query)
# conn_snow.commit()

df_info = df_info.fillna('null')
df_info = df_info.astype(str)

data_insert = [tuple(x) for x in df_info.itertuples(index=False)]

insert_query = """
    INSERT INTO SDX_GERENCIA_CRESCIMENTO.CONHECIMENTO_CLIENTE_SILVER.B042304_TI_BANCOS_BRASIL (CODIGO_ISPB, CODIGO_BANCO, NOM_BANCO)
    VALUES (%s,%s,%s)
"""

cur_snow.executemany(insert_query, data_insert)
# conn_snow.commit()
